# DT Assignment1

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제

In [3]:
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [16]:
def get_gini(df, label):
    
    at_class = df[label].value_counts() #전달받은 데이터프레임의 index, value 출력 (at_class :: attribute의 class라는 뜻)
    
    if len(at_class) == 1: #yes or no 중 하나가 없는 경우의 예외처리 ex) yes만 있는 데이터의 경우 temp['no']는 존재하지 않는 값이 되기 때문에 0으로 처리해주기 위함.
        if at_class.index == 'yes': #yes or no 중 하나가 없는 상황 중 존재하는 게 yes일 때
            at_class['no'] = 0 # temp['no']값을 0으로 처리해줌.
        else: #yes or no 중 하나가 없는 상황 중 존재하는 게 no일 때
            at_class['yes'] = 0 # temp['yes'] 값을 0으로 처리해줌.
    
    gini = 1 - (pow(at_class['yes']/len(df), 2)) - (pow(at_class['no']/len(df), 2)) # Gini(Di)계산
    
    return gini

In [17]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [20]:
import itertools # 변수의 모든 클래시 조합을 얻기 위해 itertools 불러오기
def get_binary_split(df, attribute): 
    result = [] #결과를 받을 리스트 정의
    at_class = df[attribute].value_counts().index #temp는 클래스의 인덱스를 리스트 형태로 반환받음. ex) 'age'의 경우 'youth', 'middle_aged', 'senior'
    
    for i in range(1, len(at_class)): #조합은 클래스의 개수-1의 기준으로 생성됨. ex) class가 4개라면 하나(A, B, C, D), 둘(AB, CD,...) 셋(ABC, BCD...) 개의 조합이 생성
        at_list = list(itertools.combinations(at_class, i)) #i개를 기준으로 하는 조합을 생성하고 t_list에 리스트 형태로 반환
        
        for i in range(len(at_list)): #이때 at_list는 각 조합에 대해 튜플 형태로 저장되어 있기 때문에 이를 하나 하나 받아 리스트형태로 반환하여 result에 저장
            result.append(list(at_list[i]))
    
    return result

In [21]:
get_binary_split(pd_data, "age")

[['senior'],
 ['youth'],
 ['middle_aged'],
 ['senior', 'youth'],
 ['senior', 'middle_aged'],
 ['youth', 'middle_aged']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.
- 결과로 나온 Dictionary의 Key 값은 해당 class 들로 이루어진 tuple 형태로 들어가 있습니다.

In [8]:
def get_attribute_gini_index(df, attribute, label):
    
    result = get_binary_split(df, attribute) #attribute의 모든 클래스의 조합을 함수로부터 반환받음.
    at_class = df[attribute].value_counts().index #attribute의 클래스 정보 ex) 'age'의 경우 'youth', 'middle_aged', 'senior'
    res = {} #결과를 저장할 사전형 함수 선언
    
    for i in range(len(result)): #result에 저장되어 있는 모든 클래스의 조합에 대해 한번씩 돌자
    
        comple_result = at_class.drop(result[i]) #result[i]를 제외한 조합을 반환  ex)'age'의 'youth'가 result[i]로 들어온 경우 comple_result = ['middle_aged', 'senior']
        
        q_data = df[df[attribute].isin(result[i])] #reesult[i] 조합에 대해 필터링한 데이터 셋        
        p_data = df[df[attribute].isin(comple_result)] #위에서 구한 result[i]를 제외한 나머지 comple_result 조합에 대해 필터링한 데이터 셋   
        
        gini_num1 = get_gini(q_data, label) # result[i]에 대한 지니계수       
        num1 = len(q_data) / len(df) #그때의 weight
        
        gini_num2 = get_gini(p_data, label) # comple_result에 대한 지니계수
        num2 = len(p_data) / len(df) #그때의 weight
        
        gini_index = gini_num1 * num1 + gini_num2 * num2 #최종 :: 지니계수!!
        res.update({tuple(result[i]) : gini_index}) #이를 결과를 저장할 사전형 변수에 key, value 형태로 저장해주자
        
    return res

In [9]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")
# v ^---------^ v 

{('senior',): 0.4571428571428572,
 ('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior', 'youth'): 0.35714285714285715,
 ('senior', 'middle_aged'): 0.3936507936507937,
 ('youth', 'middle_aged'): 0.4571428571428572}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

결과를 확인해보도록 하겠습니다.

In [10]:
my_dict = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - ('middle_aged',) : 0.35714285714285715


# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [25]:
##문제1 답안
income = get_attribute_gini_index(pd_data, "income", "class_buys_computer")
key_min = min(income.keys(), key=(lambda k: income[k])) #key_min :: 최솟값의 key값
print('Min -',key_min, ":", income[key_min]) #high로 나누는 것이 가장 좋아 보입니다.

('high',)
Min - ('high',) : 0.4428571428571429


In [46]:
##문제2 답안
income = get_attribute_gini_index(pd_data, "income", "class_buys_computer")
key_income = min(income.keys(), key=(lambda k: income[k]))

age = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
key_age = min(age.keys(), key=(lambda k: age[k]))

student = get_attribute_gini_index(pd_data, "student", "class_buys_computer")
key_student = min(student.keys(), key=(lambda k: student[k]))

credit_rating = get_attribute_gini_index(pd_data, "credit_rating", "class_buys_computer")
key_credit = min(credit_rating.keys(), key=(lambda k: credit_rating[k]))

imp = {key_income : income[key_income],
       key_age : age[key_age],
       key_student : student[key_student],
       key_credit : credit_rating[key_credit]}

print("<각 Feature에 대한 Min-GiniIndex> \n", imp)
print("\n")

key_min = min(imp.keys(), key=(lambda k: imp[k]))
print('Min -',key_min, ":", imp[key_min]) 
#가장 중요한 변수는 age이며 이 중에서도 middle_aged로 나누었을 때 가장 작은 값인 것을 알 수 있다.

<각 Feature에 대한 Min-GiniIndex> 
 {('high',): 0.4428571428571429, ('middle_aged',): 0.35714285714285715, ('yes',): 0.3673469387755103, ('fair',): 0.42857142857142855}


Min - ('middle_aged',) : 0.35714285714285715


In [57]:
##문제3 답안
middle_data = pd_data[pd_data['age'].isin(['middle_aged'])] #middle_aged만 있는 데이터 셋      
comple_data = pd_data[pd_data['age'].isin(['senior', 'youth'])] #middle_aged 이외의 나머지가 있는 데이터 셋

print("Middle_aged : \n", middle_data)
print("\n")
print("Senior & Youth : \n", comple_data)

Middle_aged : 
             age  income student credit_rating class_buys_computer
2   middle_aged    high      no          fair                 yes
6   middle_aged     low     yes     excellent                 yes
11  middle_aged  medium      no     excellent                 yes
12  middle_aged    high     yes          fair                 yes


Senior & Youth : 
        age  income student credit_rating class_buys_computer
0    youth    high      no          fair                  no
1    youth    high      no     excellent                  no
3   senior  medium      no          fair                 yes
4   senior     low     yes          fair                 yes
5   senior     low     yes     excellent                  no
7    youth  medium      no          fair                  no
8    youth     low     yes          fair                 yes
9   senior  medium     yes          fair                 yes
10   youth  medium     yes     excellent                 yes
13  senior  medium     

In [65]:
#Middle_age를 또 뭐로 나누어 볼까?
income = get_attribute_gini_index(middle_data, "income", "class_buys_computer")
key_income = min(income.keys(), key=(lambda k: income[k]))

student = get_attribute_gini_index(middle_data, "student", "class_buys_computer")
key_student = min(student.keys(), key=(lambda k: student[k]))

credit_rating = get_attribute_gini_index(middle_data, "credit_rating", "class_buys_computer")
key_credit = min(credit_rating.keys(), key=(lambda k: credit_rating[k]))

imp = {key_income : income[key_income],
       key_student : student[key_student],
       key_credit : credit_rating[key_credit]}

print("<각 Feature에 대한 Min-GiniIndex> \n", imp)
print("\n")

key_min = min(imp.keys(), key=(lambda k: imp[k]))
print('Min -',key_min, ":", imp[key_min]) 
#middle_aged로 나눈 데이터 셋의 target이 모두 yes이므로 지니계수도 다 0
#더 이상 분할할 것이 없다!!

<각 Feature에 대한 Min-GiniIndex> 
 {('high',): 0.0, ('yes',): 0.0, ('excellent',): 0.0}


Min - ('high',) : 0.0


In [66]:
#Senior & Youth 를 또 뭐로 나누어 볼까?
income = get_attribute_gini_index(comple_data, "income", "class_buys_computer")
key_income = min(income.keys(), key=(lambda k: income[k]))

age = get_attribute_gini_index(comple_data, "age", "class_buys_computer")
key_age = min(age.keys(), key=(lambda k: age[k]))

student = get_attribute_gini_index(comple_data, "student", "class_buys_computer")
key_student = min(student.keys(), key=(lambda k: student[k]))

credit_rating = get_attribute_gini_index(comple_data, "credit_rating", "class_buys_computer")
key_credit = min(credit_rating.keys(), key=(lambda k: credit_rating[k]))

imp = {key_income : income[key_income],
       key_age : age[key_age],
       key_student : student[key_student],
       key_credit : credit_rating[key_credit]}

print("<각 Feature에 대한 Min-GiniIndex> \n", imp)
print("\n")

key_min = min(imp.keys(), key=(lambda k: imp[k]))
print('Min -',key_min, ":", imp[key_min]) 
#가장 중요한 변수는 age이며 이 중에서도 middle_aged로 나누었을 때 가장 작은 값인 것을 알 수 있다.

<각 Feature에 대한 Min-GiniIndex> 
 {('high',): 0.375, ('senior',): 0.48, ('yes',): 0.31999999999999984, ('fair',): 0.4166666666666667}


Min - ('yes',) : 0.31999999999999984


#### 문제3 답안 :  
Senior & YOuth 데이터 셋은 다음 분할 시 Student로 나누는 것이 가장 바람직하다